In [1]:
import pandas as pd
import numpy as np
import math
import numpy as np
data = pd.read_csv("/content/agaricus-lepiota.data")

In [2]:
data.rename(columns = {'p':'Label'}, inplace = True)


In [3]:
data.describe()


,Label,x,s,n,t,p.1,f,c,n.1,k,e,e.1,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
count,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123,8123
unique,2,6,4,10,2,9,2,2,2,12,2,5,4,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,t,b,s,s,w,w,p,w,o,p,w,v,d
freq,4208,3655,3244,2283,4748,3528,7913,6811,5612,1728,4608,3776,5175,4935,4463,4383,8123,7923,7487,3967,2388,4040,3148


In [4]:
data.groupby(['Label']).count()


,x,s,n,t,p.1,f,c,n.1,k,e,e.1,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
Label,,,,,,,,,,,,,,,,,,,,,,
e,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208,4208
p,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915,3915


In [5]:
data.nunique().values


array([ 2,  6,  4, 10,  2,  9,  2,  2,  2, 12,  2,  5,  4,  4,  9,  9,  1,
        4,  3,  5,  9,  6,  7])

In [6]:
def entropy_parent():
    A = data.Label.value_counts().values
    B = data.Label.unique()
    A = A / data.shape[0]
    log = np.log2(A)
    return - np.dot(A, log)
def information_gain(entropy_parent, entropy_attribute):
    return entropy_parent - entropy_attribute

In [7]:
map_attribute_to_prob = {}
columns_attr = data.columns.tolist()[1:]
num_rows, num_columns = data.shape[0] , data.shape[1] - 1
count_att = np.delete(data.nunique().values, 0)

def get_count_label_attr_value(column_name, sort_list):
    e = data.groupby(['Label', column_name]).size().e
    p = data.groupby(['Label', column_name]).size().p

    df_e = pd.DataFrame(
        {'id': e.index,
         'e' : e.values, 
         })
    df_p = pd.DataFrame(
        {'id': p.index,
        'e' : p.values
        })
    df = pd.merge(df_e, df_p, on = "id", how = "outer")

    sorterIndex = dict(zip(sort_list, range(len(sort_list))))

    df['Tm_Rank'] = df['id'].map(sorterIndex)
    df.sort_values(['Tm_Rank'], ascending = [ True], inplace=True)
    df = df.fillna(0)

    return df.e_x.values, df.e_y.values , df.id.values

for i in range(num_columns):
    value_count = data[columns_attr[i]].value_counts().values
    value_value = data[columns_attr[i]].value_counts().index
    value_prob = value_count / num_rows

    value_e, value_p, value_id = get_count_label_attr_value(columns_attr[i], value_value)
    assert len(value_count) == len(value_value), 'The size must be equal'
    assert len(value_e) == len(value_p) 
    print(value_e + value_p)

    map_attribute_to_prob[columns_attr[i]] = {
        'count' : count_att[i],
        'values' : value_value,
        'counts' : value_count,
        'probability' : value_prob,
        'e' : value_e, 
        'p' : value_p
    }
map_attribute_to_prob 

[3655. 3152.  828.  452.   32.    4.]
[3244. 2555. 2320.    4.]
[2283. 1840. 1500. 1072. 1040.  168.  144.   44.   16.   16.]
[4748 3375]
[3528. 2160.  576.  576.  400.  400.  255.  192.   36.]
[7913  210]
[6811 1312]
[5612 2511]
[1728. 1492. 1202. 1048.  752.  732.  492.  407.   96.   86.   64.   24.]
[4608 3515]
[3776. 2480. 1119.  556.  192.]
[5175 2372  552   24]
[4935 2304  600  284]
[4463. 1872.  576.  448.  432.  192.   96.   36.    8.]
[4383. 1872.  576.  512.  432.  192.   96.   36.   24.]
[8123]
[7923.   96.   96.    8.]
[7487.  600.   36.]
[3967. 2776. 1296.   48.   36.]
[2388. 1968. 1871. 1632.   72.   48.   48.   48.   48.]
[4040. 1712. 1247.  400.  384.  340.]
[3148. 2148. 1144.  832.  367.  292.  192.]


{'c': {'count': 2,
  'counts': array([6811, 1312]),
  'e': array([3008, 1200]),
  'p': array([3803,  112]),
  'probability': array([0.83848332, 0.16151668]),
  'values': Index(['c', 'w'], dtype='object')},
 'e': {'count': 2,
  'counts': array([4608, 3515]),
  'e': array([2592, 1616]),
  'p': array([2016, 1899]),
  'probability': array([0.5672781, 0.4327219]),
  'values': Index(['t', 'e'], dtype='object')},
 'e.1': {'count': 5,
  'counts': array([3776, 2480, 1119,  556,  192]),
  'e': array([1920,  720,  864,  512,  192]),
  'p': array([1856., 1760.,  255.,   44.,    0.]),
  'probability': array([0.46485289, 0.30530592, 0.13775699, 0.06844762, 0.02363659]),
  'values': Index(['b', '?', 'e', 'c', 'r'], dtype='object')},
 'f': {'count': 2,
  'counts': array([7913,  210]),
  'e': array([4016,  192]),
  'p': array([3897,   18]),
  'probability': array([0.97414748, 0.02585252]),
  'values': Index(['f', 'a'], dtype='object')},
 'k': {'count': 12,
  'counts': array([1728, 1492, 1202, 1048,  75

In [55]:
def get_result(prob, e, p, count, row = num_rows):
  prob_e = e / count
  prob_p = p / count
  prob_e[prob_e == 0] = 0.0001
  prob_p[prob_p == 0] = 0.0001

  entro_e_x = prob_e * np.log2(prob_e)
  entro_p_x = prob_p * np.log2(prob_p)
  entro_x = entro_e_x + entro_p_x 
  entro_x = - entro_x

  return entropy_parent() - np.dot(entro_x, prob)

In [56]:
for key in columns_attr:
  object_attr = map_attribute_to_prob[key]
  print(key, get_result(object_attr['probability'], object_attr['e'], object_attr['p'], object_attr['counts']))

x 0.04880239431835931
s 0.02856890404443413
n 0.03606208720669424
t 0.1925735168444549
p.1 0.9053052452947745
f 0.014162937754335747
c 0.10086947271004221
n.1 0.23007472062980627
k 0.41691664050651567
e 0.007497410519570313
e.1 0.13493560498723745
s.1 0.2848468365023823
s.2 0.2720056979811173
w 0.253697851163327
w.1 0.24126423676500075
p.2 0.0
w.2 0.02378316776964584
o 0.03844200225665462
p.3 0.31798628807166607
k.1 0.48096871848622724
s.3 0.20194207920912965
u 0.1567456564393268
